## Test the implementation of the dataset and get some basic baselines (linear classifier)

In [1]:
from datasets.SHHS_dataset_timeonly import EEGdataset
import os
os.getcwd()

'C:\\Users\\tomsm\\PycharmProjects\\thesis\\datasets'

In [5]:
ds = EEGdataset('../../thesis01/data/',1,11,window_size=3)

In [6]:
datapoint, label = ds.__getitem__(2)
print(datapoint.shape)

torch.Size([3, 1, 3000])


In [14]:
labels = ds.labels
print(labels.size(0))
for lab in range(5):
    print(f"Percentage of labels that equals {lab}: {(labels==lab).sum()/labels.size(0)}")

9900
Percentage of labels that equals 0: 0.31111112236976624
Percentage of labels that equals 1: 0.02777777798473835
Percentage of labels that equals 2: 0.3847474753856659
Percentage of labels that equals 3: 0.16343434154987335
Percentage of labels that equals 4: 0.11292929202318192


In [21]:
# Train a linear classifier to set an ultimate baseline for neural network
# create the linear model classifier
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000,tol=1e-3))
# fit (train) the classifier
X1 = np.array(ds.X1.squeeze())
clf.fit(X1, ds.labels)


C:\Users\tomsm\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:705: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier())])

In [22]:
# mean accuracy of linear classifier: 56%
clf.score(X1, ds.labels)

0.5651515151515152

In [34]:
from datasets.SHHS_dataset_timeonly import EEGdataModule
import os
print(os.getcwd())
dm = EEGdataModule("../../data/", batch_size=64, data_split=[3, 1, 1], num_patients=10)
dm.setup()
print(len(dm.eeg_train.indices))
print(len(dm.eeg_val.indices))
print(len(dm.eeg_test.indices))
print(ds.__len__())

C:\Users\tomsm\PycharmProjects\thesis02\experiments\1D_CNN
5940
1980
1980
9900


# Testing of more efficient dataset

In [6]:
from datasets.SHHS_dataset_timeonly import SHHS_dataset
import os
import numpy as np
print(os.getcwd())

C:\Users\tomsm\PycharmProjects\thesis\datasets


In [4]:
index_file_path = '../../thesis01/data/patient_mat_list.txt'
with open(index_file_path, 'r') as f:
    index_file = f.readlines()

In [7]:
paths = list()
index = list()
for line in index_file:
    path, idx = line.split('-')
    idx = int(idx)
    assert idx > 1
    paths.append(path)
    index.append(idx)
index_np = np.cumsum(np.asarray(index))

In [9]:
import sys
sys.getsizeof(paths)

47160

In [13]:
print('Index: ', index[0:4])
print('Index np: ', index_np[0:4])

Index:  [1081, 689, 1049, 865]
Index np:  [1081 1770 2819 3684]


In [16]:
item = 2000
patient_idx = np.argmax(index_np > item)
print(patient_idx)
print(index[patient_idx])

2
1049


In [6]:
import h5py
import torch

In [9]:
import numpy as np
datapoint = '../../thesis01/data/n0001_eeg.mat'
f = h5py.File(datapoint, 'r')
X1 = torch.as_tensor(np.array(f.get("X1")))
X1.shape

torch.Size([3000, 1081])

In [33]:
item_in_patient = item - index_np[patient_idx-1]
window_size = 8
print(item_in_patient)
print([i for i in range(item_in_patient, item_in_patient+window_size)])

230
[230, 231, 232, 233, 234, 235, 236, 237]


In [37]:
print(item_in_patient)
print(index_np[patient_idx]-item)

230
819


In [44]:
X1 = torch.as_tensor(np.array(f.get("X1")))
X1 = X1[None,:,item_in_patient:item_in_patient+window_size].permute(2,0,1)
X1.shape


torch.Size([8, 1, 3000])

In [50]:
X1 = torch.as_tensor(np.array(f.get("X1")))
print(X1.shape)
X1[None, :, -window_size:].permute(2, 0, 1).shape

torch.Size([8, 1, 3000])

In [1]:
from datasets.SHHS_dataset_timeonly import SHHS_dataset

In [2]:
print('Index: ', index[0:4])
print('Index np: ', index_np[0:4])

NameError: name 'index' is not defined

In [2]:
ds = SHHS_dataset('../../thesis01/data', 1, 2, window_size=3)
ds.__len__()

2603

In [23]:
import torch
t = ds.__getitem__(700)[0]
t = t[None,:]
print(t.shape)
t = torch.squeeze(t, dim=2)
print(t.shape)

torch.Size([1, 3, 1, 3000])
torch.Size([1, 3, 3000])


In [24]:
t.shape[1]

3

In [4]:
666*3

1998